<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/unified_ai_meta_reflection_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# unified_ai/meta/reflection.py
from __future__ import annotations
from typing import Callable, Dict, Any

def reflect_on_episode(
    agent_log: str,
    prompt_template: str,
    llm_fn: Callable[[str], str] | None = None,
    parse_fn: Callable[[str], Dict[str, Any]] | None = None,
) -> Dict[str, Any]:
    if "{log}" not in prompt_template:
        raise ValueError("prompt_template must contain '{log}'")

    prompt = prompt_template.format(log=agent_log)

    def _default_llm(p: str) -> str:
        lines = [ln.strip() for ln in p.splitlines() if ln.strip()]
        failures = [ln for ln in lines if any(w in ln.lower() for w in ["fail", "error", "dead-end", "stale"])]
        if not failures:
            failures = ["No explicit failure lines detected; reward curve suggests suboptimal exploration."]
        hyp = [
            "Insufficient situational memory caused plan drift.",
            "Under-exploration due to premature exploitation."
        ]
        imps = [
            "Increase retrieval top-k and apply recency weighting in writes.",
            "Boost epsilon temporarily after failures; decay when success stabilizes."
        ]
        return (
            "Failures:\n- " + "\n- ".join(failures[:3]) + "\n\n"
            "Hypotheses:\n- " + "\n- ".join(hyp) + "\n\n"
            "Improvements:\n- " + "\n- ".join(imps)
        )

    def _default_parse(s: str) -> Dict[str, Any]:
        out = {"failures": [], "hypotheses": [], "improvements": [], "raw": s}
        sec, buf = None, []
        def _flush():
            nonlocal sec, buf
            if sec and buf:
                out[sec] = [b.lstrip("- ").strip() for b in buf if b.strip()]
                buf = []
        for ln in s.splitlines():
            l = ln.strip()
            if not l: continue
            low = l.lower()
            if low.startswith("failures:"):
                _flush(); sec = "failures"
            elif low.startswith("hypotheses:"):
                _flush(); sec = "hypotheses"
            elif low.startswith("improvements:"):
                _flush(); sec = "improvements"
            else:
                buf.append(l)
        _flush()
        return out

    raw = (_default_llm if llm_fn is None else llm_fn)(prompt)
    return (_default_parse if parse_fn is None else parse_fn)(raw)

REFLECT_PROMPT = """You are a meta-cognitive agent.

Below is a transcript of your actions, states, and outcomes during the task.

Please:
1. Identify at least 2 failure points or suboptimal decisions.
2. Hypothesize why these failures occurred.
3. Propose improvements to policy, memory use, or planning strategy.

Transcript:
{log}
"""